In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import os

f:\LangGraph-Learning\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
load_dotenv()

True

In [4]:
model = ChatOpenAI(api_key=os.getenv("OPENROUTER_API_KEY"), 
                   base_url="https://openrouter.ai/api/v1", 
                   model="openai/gpt-oss-120b:free")

In [5]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [6]:
def create_outline(state: BlogState) -> BlogState:

    #fetch the title from the state
    title= state["title"]

    # calling the llm for generating the outline
    outline_prompt = f"Create a detailed outline for a blog post titled {title}"
    outline= model. invoke(outline_prompt).content

    # updating the state with the outline
    state["outline"]=outline

    return state

In [8]:
def create_blog(state: BlogState) -> BlogState:

  #fetching the title and outline from the state
  title =state['title']
  outline= state['outline']

  prompt= f"Write a detailed blog post on the title- {title} based on the following outline: {outline}"
  content= model.invoke(prompt).content

  state["content"]= content

  return state

In [9]:
graph= StateGraph(BlogState)

graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)


graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

In [10]:
workflow= graph.compile()

In [11]:
initial_state= {"title": "The Future of AI in Everyday Life"}

final_state= workflow.invoke(initial_state)

print("Generated Blog Content:", final_state["content"])

Generated Blog Content: **The Future of AI in Everyday Life**  
*How intelligent machines are quietly reshaping the way we live, work, and play*  

---

## 1. Hook & Introduction  

It’s 6:30 a.m. and the soft glow of your bedroom lights lifts just enough to coax you out of sleep. As you stretch, the thermostat already knows you’re about to head for a jog and nudges the temperature down to a crisp 65 °F. Your coffee maker, perched on the kitchen counter, starts brewing a latte the moment you step into the hallway—its timing calibrated to the exact moment you usually reach for a mug. Outside, a sleek, driver‑less pod glides silently to the curb, doors opening as you approach, ready to whisk you to the office while you sip your coffee and skim the morning headlines on a holographic display projected onto the windshield.

That scene feels like science‑fiction, yet the building blocks are already here. In the last five years we’ve seen **machine‑learning models shrink enough to run on a ch

In [13]:
print(final_state['outline'])

**Blog Post Outline – “The Future of AI in Everyday Life”**

---

### 1. Hook & Introduction  
- **Opening anecdote or vivid scenario** – e.g., waking up to a home that anticipates your needs, a car that drives you to work while you sip coffee.  
- **Why it matters now** – rapid advances in machine learning, edge computing, and data availability.  
- **Thesis statement** – AI is moving from niche tools to an invisible, supportive layer woven into daily routines, reshaping how we live, work, and interact.  
- **Preview of main sections** – brief bullet list of the domains you’ll explore.

---

### 2. The Current Landscape: Where AI Already Lives  
| Domain | Everyday Example | Core AI Tech | Impact Today |
|--------|------------------|--------------|--------------|
| Smart Home | Voice assistants, thermostats, lighting | NLP, reinforcement learning | Energy savings, convenience |
| Personal Devices | Photo organization, predictive keyboards | Computer vision, language models | Time‑savi